# --- Day 1: Sonar Sweep ---

### --- Part One ---

In [1]:
with open("day1-input1.txt", 'r') as file:
    inputs = [int(line.replace('\n', '')) for line in file.readlines()]

In [2]:
inputs[-5:]

In [3]:
counter = 0
for i in range(1, len(inputs)):
    if inputs[i] > inputs[i-1]:
        counter += 1

In [4]:
counter

### --- Part Two ---

In [5]:
counter = 0
for i in range(1, len(inputs)-2):
    A = sum(inputs[i-1:i+2])
    B = sum(inputs[i:i+3])
    if B > A:
        counter += 1

In [6]:
counter

# --- Day 2: Dive! ---

### --- Part One ---

In [7]:
with open("day2-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]

In [8]:
depth = 0
horizontal = 0

for direction, v in [move.split() for move in inputs]:
    if direction == "forward":
        horizontal += int(v)
    elif direction == "down":
        depth += int(v)
    elif direction == "up":
        depth -= int(v)

In [9]:
depth * horizontal

1507611

### --- Part Two ---

In [10]:
depth = 0
horizontal = 0
aim = 0

for direction, v in [move.split() for move in inputs]:
    if direction == "forward":
        horizontal += int(v)
        depth += aim * int(v)
    elif direction == "down":
        aim += int(v)
    elif direction == "up":
        aim -= int(v)

In [11]:
depth * horizontal

1880593125

# --- Day 3: Binary Diagnostic ---

### --- Part One ---

In [12]:
with open("day3-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]

In [13]:
cols_num = len(inputs[0])
rows_num = len(inputs)
print("Cols:", cols_num, "Rows:", rows_num)


Cols: 12 Rows: 1000


In [14]:
import numpy as np

diagnostic_data = np.asarray([int(cell) for row in inputs for cell in row]).reshape(1000,12)

In [15]:
diagnostic_data[:5]

array([[0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0]])

In [16]:
gamma = ''
for i in range(12):
    gamma += '1' if diagnostic_data[:,i].sum() > rows_num/2 else '0'
epsilon = ''.join(['0' if v == '1' else '1' for v in gamma]) 
print("  gamma: ", gamma)
print("epsilon: ", epsilon)

  gamma:  011100101100
epsilon:  100011010011


In [17]:
int(gamma, 2) * int(epsilon, 2)

4147524

### --- Part Two ---

In [18]:
def rating_calc(data, col=0, mode='oxygen'):
    if len(data) == 1:
        return data[0]
    # for oxygen:
    keep_val =  1 if data[:,col].sum() >= len(data)/2 else 0
    if mode == 'co2':
        keep_val =  1 - keep_val
    data = data[data[:,col] == keep_val]
    return rating_calc(data, col+1, mode)

In [19]:
oxygen_rating = rating_calc(diagnostic_data, mode='oxygen')
co2_rating = rating_calc(diagnostic_data, mode='co2')
print(oxygen_rating, co2_rating)

[0 1 0 1 1 0 0 1 0 0 1 1] [1 0 0 1 1 1 0 0 0 1 1 0]


In [20]:
# convert rating to binary string and then convert to int
get_rating = lambda x: int(''.join([str(v) for v in x]), 2)

In [21]:
get_rating(oxygen_rating) * get_rating(co2_rating) 

3570354

# --- Day 4: Giant Squid ---

### --- Part One ---

In [22]:
with open("day4-numbers.txt", 'r') as file:
    numbers = file.read()

In [23]:
numbers = [int(number) for number in numbers.split(',')]

In [24]:
with open("day4-boards.txt", 'r') as file:
    boards_raw = file.read().split('\n\n')

In [25]:
boards = [list(map(int, board.replace('\n', ' ').replace('  ', ' ').strip().split(' ')))
          for board in boards_raw]

In [26]:
def check_bingo(boards):
    pattern = np.asarray([True,True,True,True,True])
    horizontal = np.where((boards==pattern).all(axis=2))[0]
    vertical = np.where((boards==pattern.reshape(5,1)).all(axis=1))[0]
    return np.append(horizontal, vertical)

In [27]:
def calc_score(bingo, boards_np, boards_np_01, last_number):
    winning_board_01 = boards_np_01[bingo]
    winning_board_01_inv = np.logical_not(winning_board_01)
    winning_board = boards_np[bingo]
    final_score = winning_board[winning_board_01_inv].sum() * last_number
    return final_score

In [28]:
import numpy as np

def play_bingo(boards, get_best=True):
    boards_np = np.asarray(boards).reshape(-1,5,5)
    boards_np_01 = np.zeros_like(boards_np)
    for i in numbers:
        boards_np_01 += boards_np == i
        bingo = check_bingo(boards_np_01)
        if len(bingo) > 0:
            last_board_score = calc_score(bingo, boards_np, boards_np_01, i)
            if get_best:
                print("BINGO!!!")
                break;
            # set values to something that will prevent these boards from further processing
            boards_np_01[bingo] = 0
            boards_np[bingo] = -1
    return last_board_score

In [29]:
play_bingo(boards)

BINGO!!!


31424

### --- Part Two ---

In [30]:
play_bingo(boards, get_best=False)

23042

# --- Day 5: Hydrothermal Venture ---

### --- Part One ---

In [31]:
with open("day5-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]
    inputs = [line.replace(' -> ', ',').split(',') for line in inputs]

In [32]:
# select just horizontal and vertical lines
inputs_hor_and_vert = [coords for coords in inputs
                      if coords[0]==coords[2] or coords[1]==coords[3]]

In [33]:
import itertools

def get_range(v1, v2):
    pts_range = list(range(min(v1, v2), max(v1, v2) + 1))
    if v1 > v2:
        # range needs to be in correct order (important for diagonals)
        pts_range = list(reversed(pts_range))
    return pts_range

def line2pts(x1, y1, x2, y2):
    x_range = get_range(x1, x2)
    y_range = get_range(y1, y2)

    if len(x_range) != len(y_range):
        # this is for horizontal and vertical lines
        pts = list(itertools.product(x_range, y_range))
    else:
    # for 45 degrees diagonals
        pts = zip(x_range, y_range)

    return pts

In [34]:
import collections

def calculate_danger_pts(inputs):
    pts_list = []
    for coords in inputs:
        coords = [int(coord) for coord in coords]
        pts_list += line2pts(*coords)

    duplicate_pts = [
        item for item, count in
        collections.Counter(pts_list).items()
        if count > 1
    ]
    return len(duplicate_pts)

In [35]:
calculate_danger_pts(inputs_hor_and_vert)

5197

### --- Part Two ---

In [36]:
calculate_danger_pts(inputs)

18605

# --- Day 6: Lanternfish ---

### --- Part One ---

In [37]:
with open("day6-input1.txt", 'r') as file:
    inputs = file.read()
    # ex: '[1,2,3,4,..]'
    inputs = [int(v) for v in inputs.split(',')]
    # ex: [1,2,3,4,..]

In [38]:
import numpy as np

def fish_spawner(input_fish, days):
    timers_np = np.asarray(inputs)

    for d in range(days):
        spawn_mask = timers_np == 0
        timers_np -= 1
        timers_np[spawn_mask] = 6
        timers_np = np.append(
            timers_np, 
            np.full(spawn_mask.sum(), 8))
    return len(timers_np)

In [39]:
fish_spawner(inputs, 80)

394994

### --- Part Two ---

In [40]:
import collections
import itertools

def fish_spawner_scalable(inputs, days):
    # turn inputs into dict of counts
    timer_dict = dict(itertools.product(range(9), [0]))
    counter = collections.Counter(inputs)
    for i, v in counter.items():
        timer_dict[i] += v
    # ex: { 0: 218, 1: 50, 2: 30, ..., 8: 0}
        
    for _ in range(days):
        # keep number of newborns
        newborns = timer_dict[0]
        # rotate values to the left in the dict
        for i in range(0,8):
            timer_dict[i] = timer_dict[i+1]
        # add # of fish which spawned new fish to # of fish with timer 6
        timer_dict[6] += newborns
        # assign newly spawned fish to timer 8
        timer_dict[8] = newborns
    return sum(timer_dict.values())

In [41]:
fish_spawner_scalable(inputs, 256)

1765974267455

# --- Day 7: The Treachery of Whales ---

### --- Part One ---

In [42]:
import numpy as np

with open("day7-input1.txt", 'r') as file:
    inputs = file.read()
    # ex: '[1,2,3,4,..]'
    inputs = [int(v) for v in inputs.split(',')]
    # ex: [1,2,3,4,..]
    inputs = np.asarray(inputs)

In [43]:
median = np.median(inputs)
print("Median: ", median)
np.abs(inputs - round(median)).sum()

Median:  349.0


355592

### --- Part Two ---

In [44]:
def calc_fuel(distance):
    return np.arange(1, distance+1).sum()

calc_fuel_vect = np.vectorize(calc_fuel)

In [45]:
fuel_usage_sum_list = []
for i in range(min(inputs), int(max(inputs))):
    distance_arr = np.abs(inputs - round(i))
    fuel_usage_arr = calc_fuel_vect(distance_arr)
    fuel_usage_sum_list.append(fuel_usage_arr.sum())
fuel_usage_sum_list = np.asarray(fuel_usage_sum_list)

In [46]:
fuel_usage_sum_list[fuel_usage_sum_list.argmin()]

101618069

In fact using mean should work too

In [47]:
mean = np.mean(inputs)
# calc distance arr between crab positions and target position
distance_arr = np.abs(inputs - int(mean))
# calc fuel usage for each distance (vetorized)
fuel_usage_arr = calc_fuel_vect(distance_arr)
# sum all fuel used by all crabs
fuel_usage_arr.sum()

101618069

# --- Day 8: Seven Segment Search ---

### --- Part One ---

In [48]:
import numpy as np

with open("day8-input1.txt", 'r') as file:
    all_inputs = file.readlines()

all_inputs = [line.replace('\n','').replace(' | ', '|').split('|')
          for line in all_inputs]
inputs, outputs = map(list, zip(*all_inputs))

In [49]:
def get_easy_digits(output):
    easy_digits_lens = [2, 3, 4, 7]
    signals = output.split()
    easy_digits = [signal for signal in signals
                  if len(signal) in easy_digits_lens]
    return easy_digits
    

In [50]:
sum([len(get_easy_digits(out)) for out in outputs])

479

### --- Part Two ---

In [51]:
# digits to ground truth signal
digits_sig_gt = {
    0: 'abcefg',
    1: 'cf',
    2: 'acdeg',
    3: 'acdfg',
    4: 'bcdf',
    5: 'abdfg',
    6: 'abdefg',
    7: 'acf',
    8: 'abcdefg',
    9: 'abcdfg'
}

# signal length to potential digits
sig_len_2_digits = {
    2:[1], 3:[7], 4:[4], 7:[8], 5: [2, 3, 5], 6: [6, 9, 0]
}

# signal length to possible ground truth signals
sig_len_2_gt_sig = dict(
    zip(
        sig_len_2_digits.keys(),
        [[digits_sig_gt[digit] for digit in sig_len_2_digits[length]]
         for length in sig_len_2_digits.keys()]
    )
)
sig_len_2_gt_sig

{2: ['cf'],
 3: ['acf'],
 4: ['bcdf'],
 7: ['abcdefg'],
 5: ['acdeg', 'acdfg', 'abdfg'],
 6: ['abdefg', 'abcdfg', 'abcefg']}

In [52]:
def crack_mapping(signal_dict):
    # signal_dict = {'a': 'eda', 'b': 'agcd', 'c': 'ad', 'd': 'agcd', 'e': 'bgefdac', 'f': 'ad', 'g': 'bgefdac'}
    temp = [item for item in signal_dict.items() if len(item[1]) <= 2]
    # temp = [[('c', 'ad'), ('f', 'ad')]]
    gt_sigs, map_sigs = map(list, zip(*temp))
    # gt_sigs = ['c', 'f']
    # map_sigs = ['ad', 'ad']
    map_sigs = np.unique(list(''.join(map_sigs)))
    # map_sigs = ['a' 'd']
    for mapping in signal_dict.items():
        # mapping = ('a', 'eda')
        if mapping[0] not in gt_sigs:
            sig = mapping[1]
            # sig = 'eda'
            for c in map_sigs:
                sig = sig.replace(c, '')
                # sig = 'e'
            signal_dict[mapping[0]] = sig
    # do recursion until each mapping point to maximum of 2 gt signals
    if len(''.join(signal_dict.values())) < 2*len(signal_dict.values()):
        return signal_dict
    else:
        return crack_mapping(signal_dict)

In [53]:
out_numbers = []
# zip inputs and outputs and iterate
for in1, out1 in zip(inputs[:], outputs[:]):
    # placeholder for signals decoder
    signal_dict = dict(zip('abcdefg', ['' for i in range(7)]))
    
    # here we join inputs with outputs to analyze them and build correct signal decoder
    # we focus only on easiest digits that we can clearly decode based on 
    # for each signal check length and match it with output digits to get ground truth signalsignal length
    signals = get_easy_digits(in1) + get_easy_digits(out1)
    for sig in signals:
        for segm in digits_sig_gt[sig_len_2_digits[len(sig)][0]]:
            if len(signal_dict[segm]) > 1:
                signal_dict[segm] = ''.join([i for i in sig if i in signal_dict[segm]])
            if len(signal_dict[segm]) == 0:
                signal_dict[segm] = sig
    signal_dict = crack_mapping(signal_dict)
    
    err_sig_2_sig_dict = dict(zip('abcdefg', ['' for i in range(7)]))
    for pair in [(c, s[0]) for s in signal_dict.items() for c in s[1]]:
        err_sig_2_sig_dict[pair[0]] += pair[1]
    
    out_number = ''
    
    for o in out1.split():
        if len(o) not in [2,3,4,7]:
            decoded = [err_sig_2_sig_dict[c] for c in o]
            decoded_clean = np.unique(decoded, return_counts=True)
            decoded_final = []
            for c in zip(*decoded_clean):
                if c[1] > 1:
                    for k in c[0]:
                        decoded_final.append(k)
                else:
                    decoded_final.append(c[0])
            decoded_final = [c for c in decoded_final if len(c) == 1]
            for sig, dig in zip(sig_len_2_gt_sig[len(o)], sig_len_2_digits[len(o)]):
                if set(decoded_final).issubset(list(sig)):
                    out_number += str(dig)
                    break
        else:
            out_number += str(sig_len_2_digits[len(o)][0])

    out_numbers.append(int(out_number))

In [54]:
sum(out_numbers)

1041746

# --- Day 9: Smoke Basin ---

### --- Part One ---

In [55]:
import numpy as np

with open("day9-input1.txt", 'r') as file:
    all_inputs = file.readlines()
all_inputs = [line.replace('\n','') for line in all_inputs]

height_map = np.asarray([int(val) for line in all_inputs
                         for val in line]
                       ).reshape(len(all_inputs), len(all_inputs[0]))
# pad with 9 for sliding window
height_map = np.pad(height_map, [[1,1],[1,1]], constant_values=9)
height_map.shape

(102, 102)

In [56]:
lowest_points = []
# sliding window
for i in range(1, height_map.shape[0]-1):
    for j in range(1, height_map.shape[1]-1):
        # take 3x3 slice
        local_slice = height_map[j-1:j+2, i-1:i+2]
        # if central value is minimum check if it is unique in slice
        if local_slice.min() == height_map[j,i]:
            # check if minimal value is also unique
            values, counts = np.unique(local_slice, return_counts=True)
            if counts[0] == 1:
                lowest_points.append((j,i))

sum([height_map[pt[0],pt[1]] + 1 for pt in lowest_points])

512

### --- Part Two ---


In [57]:
def get_basin_pts(start_point, total_basin_points, height_map):
    # recursive function to look for all basin points
    # point to check from
    x, y = start_point
    # need to select points in all adjacent horizontal + vertical directions
    pts_to_check = [(x-1,y), (x,y-1), (x+1,y), (x,y+1)]
    # then we filter those potential points to exclude 9 and duplicates
    new_basin_points = [pt for pt in pts_to_check
                        if height_map[pt] != 9 and pt not in total_basin_points]
    # if we found new points add them to total points list and continue recursion
    if len(new_basin_points) > 0:
        total_basin_points += new_basin_points
        # we continue recursion for each new basin point found
        for pt in new_basin_points:
            total_basin_points = get_basin_pts(
                pt, total_basin_points, height_map)
    # if no new points found, return total basin points
    return total_basin_points

In [58]:
all_basins = []
# iterate through all lowest points
for pt in lowest_points:
    # get basin points for low point
    basin_pts = get_basin_pts(pt, [], height_map)
    # add basin sice to collection
    all_basins.append(len(basin_pts))

# sort basin sizes (largest go last)
all_basins.sort()

all_basins[-3] * all_basins[-2] * all_basins[-1]

1600104